In [71]:
import numpy as np
import scanpy as sc
import anndata as ad
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# Import Data

In [72]:
adata = ad.read_h5ad('/data/rudensky/EYW/SIG07/scanpy_outs/SIG07_doublets_CR_RNA_zscore.h5ad')
adata.X = adata.layers['counts'].copy()

In [73]:
# replace linker-1 and linker-2 with just linker
adata.obs['ligand_call_oBC_CR'] = adata.obs['ligand_call_oBC_CR'].str.replace(
    r'linker-(1|2)', 'linker', regex=True)
print(adata.obs['ligand_call_oBC_CR'].unique())

['IL4_linker' 'IL4_IL12' 'IL12_IFNA' 'IL4_IFNA' 'IL12_linker' 'IL12_IL6'
 'IL2_TNF' 'IL21_linker' 'IFNA_linker' 'IL2_linker' 'IL2_IL6' 'IL4_IL6'
 'TNF_linker' 'IL2_IL12' 'IL2_IL4' 'IFNA_TNF' 'IL6_linker' 'IL4_IL21'
 'IL4_IL27' 'linker_linker' 'IL27_linker' 'IL27_TNF' 'IFNA_IL27'
 'IL6_IL21' 'IL2_IL27' 'IL4_TNF' 'IL6_TNF' 'IL2_IL21' 'IL6_IFNA'
 'IL6_IL27' 'IL2_IFNA' 'IL21_TNF' 'IFNA_IL21' 'IL21_IL27' 'IL12_TNF'
 'IL12_IL27' 'IL12_IL21']


In [24]:
def subset_anndata_by_groups(adata, group_a, group_b, group_column="group"):
    """
    Subset an AnnData object to only include cells belonging to specified groups
    (group_a, group_b, and their interaction group_c). Adds categorical columns
    to .obs indicating membership in each group.

    Parameters:
        adata: AnnData object
            The input AnnData object.
        group_a: str
            The name of the first group to filter.
        group_b: str
            The name of the second group to filter.
        group_column: str
            The name of the column in .obs containing group information (default: "group").
    
    Returns:
        AnnData: A subsetted AnnData object containing cells belonging to group_a, group_b, and group_c.
    """
    # Derive group C
    ligands = sorted([group_a, group_b])
    possible_group_c = [f"{ligands[0]}_{ligands[1]}", f"{ligands[1]}_{ligands[0]}"]
    
    # Check which of the possible group C values exists in the dataset
    group_c = next((group for group in possible_group_c if group in adata.obs[group_column].unique()), None)
    if group_c is None:
        raise ValueError(f"Neither '{possible_group_c[0]}' nor '{possible_group_c[1]}' found in the dataset.")
    
    # Filter for groups A, B, and C
    subset_mask = (
        (adata.obs[group_column] == f"{group_a}_linker") |
        (adata.obs[group_column] == f"{group_b}_linker") |
        (adata.obs[group_column] == group_c)
    )
    adata_subset = adata[subset_mask].copy()
    
    # Add categorical columns for group membership in .obs
    adata_subset.obs[f"is_{group_a}"] = adata_subset.obs[group_column] == f"{group_a}_linker"
    adata_subset.obs[f"is_{group_b}"] = adata_subset.obs[group_column] == f"{group_b}_linker"
    adata_subset.obs[f"is_{group_c}"] = adata_subset.obs[group_column] == group_c
    
    # Convert the new columns to categorical dtype
    for col in [f"is_{group_a}", f"is_{group_b}", f"is_{group_c}"]:
        adata_subset.obs[col] = adata_subset.obs[col].astype("category")
    
    return adata_subset

In [25]:
# Example Usage
# Assuming `adata` is your AnnData object
subset_adata = subset_anndata_by_groups(adata, "IL4", "IL27", group_column="ligand_call_oBC_CR")
# View the resulting .obs
print(subset_adata.obs.head())

                            num_ligand_oBC_CR num_umis_oBC_CR  \
cell_barcode                                                    
AAACAAGCAAACTGGCACAGACCT-1                2.0             5|5   
AAACAAGCAAGTTGCAATACGTCA-1                2.0           15|12   
AAACAAGCACTTTGTAAGCTGTGA-1                2.0             7|4   
AAACAAGCAGAAAGGTATGTTGAC-1                2.0           13|17   
AAACAAGCAGACTCAAACTACTCA-1                2.0            2|13   

                            num_unique_groups_CR ligand_call_oBC_CR  \
cell_barcode                                                          
AAACAAGCAAACTGGCACAGACCT-1                   1.0         IL4_linker   
AAACAAGCAAGTTGCAATACGTCA-1                   1.0         IL4_linker   
AAACAAGCACTTTGTAAGCTGTGA-1                   1.0           IL4_IL27   
AAACAAGCAGAAAGGTATGTTGAC-1                   1.0         IL4_linker   
AAACAAGCAGACTCAAACTACTCA-1                   1.0        IL27_linker   

                           group_call_CR oBC_c

In [57]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def compute_glimmirs_int_model_with_groups(adata_subset,target_gene,A,B):
    """
    Compute the GLiMMIRS-int model for a single gene in a subsetted AnnData object,
    using is_A, is_B, and is_A_B columns as X_A, X_B, and X_AB, respectively.
    
    Parameters:
        adata_subset: AnnData object
            Subsetted AnnData object from the `subset_anndata_by_groups` function.
    
    Returns:
        model_results: statsmodels GLMResults
            Results of the GLM model fit.
    """
    # Add a pseudocount to avoid inflation of coefficients (as described in the model)
    pseudocount = 0.01
    y = adata_subset.X[:, adata_subset.var.index == target_gene].toarray().flatten() + pseudocount

    # Use is_A, is_B, and is_A_B as X_A, X_B, and X_AB
    X_A = adata_subset.obs[f"is_{A}"].astype(int)  # Convert boolean to integer
    X_B = adata_subset.obs[f"is_{B}"].astype(int)  # Convert boolean to integer
    X_AB = adata_subset.obs[f"is_{A}_{B}"].astype(int)  # Convert boolean to integer
    
    # Extract additional predictors from adata_subset.obs
    X_S = adata_subset.obs["S_score"]  # S cell cycle state
    X_G2M = adata_subset.obs["G2M_score"]  # G2M cell cycle state
    X_mito = adata_subset.obs["pct_counts_mt"]  # Percentage of mitochondrial reads
    ln_s = np.log(adata_subset.obs["total_counts"])  # Sequencing depth scaling factor

    # Map non-numeric group_call_CR to numeric values
    group_mapping = {group: i for i, group in enumerate(adata_subset.obs["group_call_CR"].unique())}
    X_group = adata_subset.obs["group_call_CR"].map(group_mapping)  # Map groups to integers
    

    # Combine predictors into a DataFrame without an intercept column
    predictors = pd.DataFrame({
        "X_A": X_A,
        "X_B": X_B,
        "X_AB": X_AB,
        "X_S": X_S,
        "X_G2M": X_G2M,
        "X_mito": X_mito,
        "X_group": X_group,
        "ln_s": ln_s,
    })

    # Fit a GLM model with a negative binomial distribution
    model = sm.GLM(y, predictors,
                   family=sm.families.NegativeBinomial(link=sm.families.links.log()))
    results = model.fit()

    return results

In [42]:
# Example Usage
# Assuming `adata_subset` is the result of subset_anndata_by_groups
subset_adata = subset_anndata_by_groups(adata, "IL4", "IL27",group_column="ligand_call_oBC_CR")

In [55]:
results = compute_glimmirs_int_model_with_groups(subset_adata,"Il4ra","IL4","IL27")

/lila/home/wange7/miniforge3/envs/scanpy_standard/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [56]:
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                14207
Model:                            GLM   Df Residuals:                    14199
Model Family:        NegativeBinomial   Df Model:                            7
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -33035.
Date:                Wed, 27 Nov 2024   Deviance:                       7860.0
Time:                        11:50:24   Pearson chi2:                 6.74e+03
No. Iterations:                     8   Pseudo R-squ. (CS):             0.4801
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
X_A           -7.5938      0.264    -28.727      0.0

In [ ]:
df = pd.DataFrame(adata.X.toarray(), index=

subset_adata.X

In [59]:
def anndata_X_to_df(adata):
    """
    Save the `.X` matrix of an AnnData object as a CSV file with cell barcodes and column names preserved.

    Parameters:
        adata: AnnData object
            The AnnData object containing the matrix to be saved.
        output_file: str
            The path to save the CSV file.
    """
    # Convert .X to a DataFrame
    df = pd.DataFrame(
        adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X,  # Convert sparse matrix to dense if needed
        index=adata.obs.index,  # Use cell barcodes as row indices
        columns=adata.var.index  # Use feature names (e.g., gene names) as column names
    )
    
    # Save DataFrame to a CSV file
    return df

In [74]:
tiny_ad = adata[:, adata.var.index.isin(['Il4ra','Acvr1a','Irf4','Il12rb2','Metrnl'])]
temp = anndata_X_to_df(tiny_ad).to_csv("/data/rudensky/EYW/git_projects/SIG07/analysis_outs/IL4_IL27_test_counts.csv")
tiny_ad.obs.to_csv("/data/rudensky/EYW/git_projects/SIG07/analysis_outs/IL4_IL27_test_obs.csv")

In [ ]:
temp